# PPGAN notebook version for MLiP

A lot of code is reused from the Patch Permutation GAN implementation by to i-evi (https://github.com/i-evi/p2gan)
Their code is not a notebook and only works on Tensorflow 1, so most of our adjustments to this code are to make it work with TF2 as a notebook.
Credits to the authors of the Patch Permuation GAN paper (https://arxiv.org/abs/2001.07466)

## Imports
We use the tf_slim package and tensorflow.compat.v1 library to make the TF1 code from the paper work in TF2 (which is what Kaggle uses).
Furthermore, we also download the vgg16.npy weights file which you normally would manually place in the project folder.

In [ ]:
# Required slim API for tf2 and vgg16 weights file
!pip install tf_slim
!wget https://www.dropbox.com/s/8a8rei66f72um4i/vgg16.npy
    
!wget https://www.b4rt.nl/files/SUN397Landscape-ish_256x256.zip
!unzip -qq SUN397Landscape-ish_256x256.zip

# Libraries
import os
import random
import time
import json
import cv2
import inspect
import shutil
import numpy as np
from matplotlib import pyplot as plt

# Compatibility imports (to run the tf1 code in tf2)
import tf_slim as slim
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## Parameters
Originally, many of these parameters were set in a JSON config file, or placed in some of the Python modules.
Since we made a notebook version, we placed all configurable parameters here and removed the JSON configuration.

In [ ]:
# Most important parameters for our experiments
BATCH_SIZE   = 8
LAMBDA       = 0.000005 # "hyper parameter to balance content and style" default: 0.000005, higher == more content
TRAIN_EPOCHS = 10

# Image size
PSI_D_SIZE   = 256 
G_IMG_SIZE   = 256
PATCH_SIZE   = 16 # Size of patches

# VGG params
VGG_L        = 1
VGG_FEATURES = 64

# Gen/Discrim params
DISCRIMINATOR_MAX_ITER = 1
GENERATOR_MAX_ITER = 2

# Training paths
STYLE_PATH      = "/kaggle/input/gan-getting-started/monet_jpg/" # Do not forget trailing slash
TRAINSET_PATH   = "/kaggle/working/SUN397/" #"/kaggle/input/gan-getting-started/photo_jpg/" # Do not forget trailing slash

# Render paths
IMGSRC_PATH = "/kaggle/working/SUN397/" #"/kaggle/input/gan-getting-started/photo_jpg/" # path of input images to convert. Do not forget trailing slash
OUT_PATH    = "/kaggle/working/images/" # path of output images. Do not forget trailing slash

gpu_options = tf.GPUOptions(allow_growth=True)

# List of style images we want to use
#STYLE_IMAGE_LIST = [n for n in os.listdir(STYLE_PATH)]

# Custom list of hand picked style images
#STYLE_IMAGE_LIST = ["0bd913dbc7.jpg", "3b262c6726.jpg", "4c74254ad3.jpg", "6f0b9df5c5.jpg", "19dc36ccb2.jpg", "4c0e35882c.jpg", "5cb895b722.jpg", "92c0ba8c0d.jpg"]
#STYLE_IMAGE_LIST = ["e753318d04.jpg"]
STYLE_IMAGE_LIST = ["000c1e3bff.jpg", "0bd913dbc7.jpg", "0e3b3292da.jpg", "1a127acf4d.jpg", "1e4e4e63c5.jpg", "1f9667f2a7.jpg", "1f22663e72.jpg", "2acfbab228.jpg",
                    "2c00f5147f.jpg", "2cca56415e.jpg", "2e0d0e6e19.jpg", "2f90c99e10.jpg", "2f20944b6a.jpg", "3b262c6726.jpg", "3c341ff93e.jpg", "3d13fe022e.jpg",
                    "3deea9f4a4.jpg", "3eaef3ee43.jpg", "4ab2583fe2.jpg", "4ad8b366c1.jpg", "4b0adf7c6f.jpg", "4c0e35882c.jpg", "4c74254ad3.jpg", "4e05523825.jpg",
                    "4f4de0bbba.jpg", "4f7e01f097.jpg", "4f045779b0.jpg", "05b493ff42.jpg", "5aa514ffac.jpg", "5c79cfe0b3.jpg", "5cb895b722.jpg", "5ffbfe68d4.jpg",
                    "6bbe0e63c6.jpg", "6bfbd1df5b.jpg", "6c6cc46498.jpg", "6d0e87f557.jpg", "6e0429f92e.jpg", "6ee7c39dbc.jpg", "6f0b9df5c5.jpg", "07fcaee35f.jpg",
                    "7cb36714d0.jpg", "7d64c3100c.jpg", "8b54448a07.jpg", "8c48e112d0.jpg", "8c8011c291.jpg", "8e5ff15568.jpg", "8f02369f42.jpg", "9f409e3376.jpg",
                    "9fc868e864.jpg", "10c555c1b1.jpg", "11ab570c5e.jpg", "19dc36ccb2.jpg", "22b1ac6b44.jpg", "23f0fbd77e.jpg", "24af733334.jpg", "25c9904782.jpg",
                    "26b66eb819.jpg", "32cc820303.jpg", "32e33792cc.jpg", "40d7d18ad3.jpg", "049e293b93.jpg", "052a77c020.jpg", "52aed0f943.jpg", "52d12dc627.jpg",
                    "52fc351abf.jpg", "058f878b7c.jpg", "59df696966.jpg", "61e735361a.jpg", "066fe4cbaa.jpg", "66a144f547.jpg", "68b60c04b7.jpg", "68d60af226.jpg",
                    "69f4b75a37.jpg", "73f33a12c5.jpg", "74e452fb31.jpg", "76cc7181f8.jpg", "77b37629f2.jpg", "82b9fd68b1.jpg", "89d970411d.jpg", "89fcbf2f76.jpg",
                    "92c0ba8c0d.jpg", "95a53d7b0b.jpg", "95b5f01a85.jpg", "99a51d3e25.jpg", "99d94af5dd.jpg", "118da0690c.jpg", "133b42e498.jpg", "184d6c66cd.jpg",
                    "252d9a4abc.jpg", "281b73fb5e.jpg", "344d1829bb.jpg", "417e77e946.jpg", "429e382095.jpg", "512cd171a9.jpg", "526a110636.jpg", "536aa87152.jpg",
                    "565f19268b.jpg", "586acab7c5.jpg", "593db29cce.jpg", "608ee0d370.jpg", "632ddbc784.jpg", "661e374153.jpg", "676a5a4c2e.jpg", "730f325b64.jpg",
                    "853f8d711f.jpg", "893db2701d.jpg", "932d0dd808.jpg", "990ed28f62.jpg", "1814cc6632.jpg", "2759c1ed37.jpg", "3843e221cc.jpg", "4995c04b1a.jpg",
                    "05144e306f.jpg", "5185e8c56a.jpg", "5926f85cbf.jpg", "6043aadea0.jpg", "7017e6caa1.jpg", "7239ba0b55.jpg", "7341d96c1d.jpg", "7960adbd50.jpg",
                    "8114fa2607.jpg", "8314acfd35.jpg", "9843bc25c5.jpg", "9908d1daa9.jpg", "011835cfbf.jpg", "14162de938.jpg", "17557a29cb.jpg", "23832dead5.jpg",
                    "29696b4455.jpg", "49337b68f4.jpg", "064487d630.jpg", "66226e18fc.jpg", "68729aac07.jpg", "79224da51f.jpg", "79292e1434.jpg", "82991e742a.jpg",
                    "89964efa86.jpg", "93132f89ee.jpg", "106757e5d8.jpg", "463835bbc6.jpg", "488600cb75.jpg", "695897bd4a.jpg", "718445ebe3.jpg", "910610e827.jpg",
                    "910729e0ce.jpg", "1078363ff0.jpg", "2581464ddc.jpg", "4660310c3e.jpg", "7952021d2f.jpg", "85580214be.jpg", "88402296cc.jpg", "158740962c.jpg",
                    "599098859e.jpg", "815624563e.jpg", "3545597386.jpg", "6742294320.jpg", "7054793632.jpg", "a4e4a61fb2.jpg", "a8fbbe3eb1.jpg", "a030bc32e6.jpg",
                    "a59f3f5b89.jpg", "a96b79a93f.jpg", "a210ceedc7.jpg", "a885da7b52.jpg", "a6291c2a1c.jpg", "a7977705be.jpg", "ad8ce41fc0.jpg", "b1ea5d5a7d.jpg",
                    "b2ce76c750.jpg", "b3adc75e7d.jpg", "b5c2fe7c4c.jpg", "b13c0973ee.jpg", "b44f24c048.jpg", "b76d52e05a.jpg", "b256e61a5d.jpg", "b1310da865.jpg",
                    "b99546090b.jpg", "ba52f976af.jpg", "baf6efabfe.jpg", "bbc5ac4564.jpg", "bc4b364a44.jpg", "bf6db09354.jpg", "c1dc1a85a4.jpg", "c6c88ce9c4.jpg",
                    "c6c360756c.jpg", "c7d8142152.jpg", "c67ba2060c.jpg", "c68c52e8fc.jpg", "c4622e3fb6.jpg", "c14505c1da.jpg", "c2576267d4.jpg", "cb9c553ded.jpg",
                    "cc2bb659f4.jpg", "cd6623d07d.jpg", "ceb6cf5f31.jpg", "cfc6fce7b5.jpg", "d05cab011d.jpg", "d6d6e625bd.jpg", "d14c1abdd4.jpg", "d239dae42d.jpg",
                    "d087730b76.jpg", "d754850d01.jpg", "d88482796d.jpg", "da72006ef5.jpg", "dc33f0edbe.jpg", "dd46691bd7.jpg", "de6f71b00f.jpg", "df64ac2dcb.jpg",
                    "e9f686534b.jpg", "e88d9de918.jpg", "e510a74d3c.jpg", "e568f84fad.jpg", "e2253b87a0.jpg", "e9580cd500.jpg", "e37407c747.jpg", "e753318d04.jpg",
                    "e3112413b1.jpg", "eb3cc5c559.jpg", "ec78d80dbd.jpg", "ec3398cef9.jpg", "ed597655a0.jpg", "ede9769cb3.jpg", "ee7adac58f.jpg", "f0d789c4bc.jpg",
                    "f486c1655f.jpg", "f0884db067.jpg", "f821791c85.jpg", "fb3b06dcb2.jpg", "fb806a2a1c.jpg", "fb93438ff9.jpg", "fba982625d.jpg", "fc11d52502.jpg",
                    "fd63a333f1.jpg", "ffd74c77ea.jpg"]

STYLE_IMAGE_COUNT = len(STYLE_IMAGE_LIST)

## UTIL functions
Contain functions for loading images, creating patches, etc.
Mostly unchanged from util.py from the original code. At the end of this code block we made a special function that allows the model to work with multiple style images (instead of just one)

In [ ]:
def imbatch_read(path, file_list, shape):
    if len(file_list) <= 0:
        return None
    sls = []
    batch = None
    for i, item in enumerate(file_list):
        img = cv2.imread(path +  item)
        sls.append((img.shape[1], img.shape[0]))
        img = cv2.resize(img, shape)
        img = (img / 255.0 - 0.5) * 2.0
        img = np.expand_dims(img, axis=0)
        if not isinstance(batch, np.ndarray):
            batch = img
        else:
            batch = np.vstack((batch, img))
    return batch, sls

def img_write(img, path, shape):
    if len(img.shape) != 3 or img.shape[2] != 3:
        return False # Only support RGB img
    img = (((img + 1) / 2) * 255)#.astype(np.uint8)
    img = cv2.resize(img, shape)
    cv2.imwrite(path, img)
    return True

def background(shape, ch=3):
    return np.zeros(shape[0] * shape[1] * ch).\
        astype(np.uint8).reshape((shape[0], shape[1], ch))

def crop(img, x, y, xl, yl):
    if (x + xl) > img.shape[1] or (y + yl) > img.shape[0]:
        return None
    return img[y:y+yl, x:x+xl,:]

def merge(bg, img, x, y):
    bg[y:y+img.shape[0] if y+img.shape[0] < bg.shape[0] else bg.shape[0],
        x:x+img.shape[1] if x+img.shape[1] < bg.shape[1] else bg.shape[1],:] =\
            img[:img.shape[0] if y+img.shape[0] < bg.shape[0] else bg.shape[0] - y,
        :img.shape[1] if x+img.shape[1] < bg.shape[1] else bg.shape[1] - x,:]

def random_crop(img, xl, yl):
    y = int((img.shape[0] - yl) * random.random())
    x = int((img.shape[1] - xl) * random.random())
    return crop(img, x, y, xl, yl)

def patch_fill(bg, img, offset_x, offset_y, gap, s):
    if bg.shape[0] != bg.shape[1]:
        exit('Error filling patches, background or image must be square')
    # if s % 2 != 1:
    #   exit('Patch size must be odd, not %d'%s)
    jump = s + gap
    cnt = int(bg.shape[0] / (jump))
    for j in range(cnt):
        for i in range(cnt):
            merge(bg, random_crop(img, s, s),
                i * jump + offset_x, j * jump + offset_y)
    return bg

def patch_fill_multi(bg, imgls, offset_x, offset_y, gap, s):
    if bg.shape[0] != bg.shape[1]:
        exit('Error filling patches, background or image must be square')
    # if s % 2 != 1:
    #   exit('Patch size must be odd, not %d'%s)
    jump = s + gap
    cnt = int(bg.shape[0] / (jump))
    for j in range(cnt):
        for i in range(cnt):
            merge(bg, 
                random_crop(imgls[int(random.random() * len(imgls))], s, s),
                i * jump + offset_x, j * jump + offset_y)


def pickup_list(ls, cnt, begin):
    ls_len = len(ls)
    if begin >= ls_len:
        return None
    if (begin + cnt) >= ls_len:
        begin = ls_len - cnt
    return ls[begin:begin+cnt]


def pickup_list_random(ls, batchSize):
    return random.sample(ls, batchSize)


def make_input_batch(img, bs, h, w, ps):
    (_, _, c) = img.shape
    bat = np.zeros(bs * h * w * c).astype(np.float32).reshape((bs, h, w, c))
    for i in range(bs):
        patch_fill(bat[i], img, 0, 0, 0, ps)
    return bat


def make_input_batch_multi(imgls, bs, h, w, ps):
    (_, _, c) = imgls[0].shape
    bat = np.zeros(bs * h * w * c).astype(np.float32).reshape((bs, h, w, c))
    for i in range(bs):
        patch_fill_multi(bat[i], imgls, 0, 0, 0, ps)
    return bat

def ls_files_to_json(path, jname=None, reverse=False, ext=[]):
    ls = []
    for x in os.listdir(path):
        if os.path.isfile(os.path.join(path, x)):
            if len(ext) == 0:
                ls.append(x)
            else:
                if x.split('.')[-1] in ext:
                    ls.append(x)
    if reverse == None:
        pass
    elif reverse == False:
        ls.sort(reverse=False)
    elif reverse == True:
        ls.sort(reverse=True)
    if isinstance(jname, str):
        with open(jname, 'w+') as fp:
            fp.write(json.dumps(ls))
    return ls

def open_img(img_path):
    img = cv2.imread(img_path)
    img = (img / 255.0 - 0.5) * 2.0
    return img

def load_bat_img(img_path, shape, prep=True, singleCh=False, gray=False, remove_pad=False):
    img = cv2.imread(img_path)
    if gray:
        img= cv2.cvtColor(img,cv2.COLOR_RGB2GRAY).reshape((img.shape[0], img.shape[1], 1))
        timg = np.copy(img)
        img = np.concatenate((img, timg), axis=2)
        img = np.concatenate((img, timg), axis=2)
    if remove_pad:
        y = img.shape[0]
        x = img.shape[1]
        img = img[int(y/8):y - int(y/8), int(x/8):x-int(x/8),:]
    if isinstance(shape, tuple):
        img = cv2.resize(img, shape)
    img = np.expand_dims(img, axis=0)
    if prep:
        img = (img / 255.0 - 0.5) * 2.0
    if singleCh:
        img = np.mean(img, axis=3)
        img = np.expand_dims(img, axis=3)
    return img

def random_file_list(path, batch_size, rand=True):
    images_path =[n for n in os.listdir(path)]
    if (batch_size > len(images_path) or batch_size <= 0):
        print('N/A')
        return None
    if rand:
        random.shuffle(images_path)
    return images_path[0:batch_size]

def images_batch(folder, file_list, shape, prep=True, singleCh=False, gray=False, remove_pad=False):
    if len(file_list) <= 0:
        return None
    batch = load_bat_img(folder+'/'+file_list[0], prep=prep, shape=shape, singleCh=False, gray=gray, remove_pad=remove_pad)
    for i in range(len(file_list) - 1):
        batch = np.vstack((batch, load_bat_img(folder+'/'+file_list[i + 1], prep=prep, shape=shape, singleCh=False, gray=gray, remove_pad=remove_pad)))
    return batch

def save_as_rgb_img(img, path):
    if len(img.shape) != 3 or img.shape[2] != 3:
        return False # Only support RGB img
    img = (((img + 1) / 2) * 255)
    cv2.imwrite(path, img)
    return True

def save_batch_as_rgb_img(bat, path, prefix=None):
    code = 0
    for item in bat:
        if prefix == None:
            save_as_rgb_img(item, path+'/%d.jpg'%code)
        else:
            save_as_rgb_img(item, path+'/%s%d.jpg'%(prefix, code))
        code = code + 1

def random_sub_set(set, n):
    if n <= 1:
        return set
    set_index = np.arange(1, set.shape[0] + 1)
    subset_index = np.random.choice(set_index, n)
    batch_data = set10[set10_index[0]]
    batch_data = np.expand_dims(batch_data, axis=0)
    for item in subset_index[1:]:
        batch_data = np.vstack((batch_data, np.expand_dims(set10[item], axis=0)))
    return batch_data

def sub_set(inset, index_tab):
    if len(index_tab) <= 1:
        return inset
    batch_data = inset[index_tab[0]]
    batch_data = np.expand_dims(batch_data, axis=0)
    for item in index_tab[1:]:
        batch_data = np.vstack((batch_data, np.expand_dims(inset[item], axis=0)))
    return batch_data

def silent_mkdir(path):
    try:
        os.makedirs(path)
    except:
        pass

############################
# CUSTOM MULTI STYLE VERSION
############################
# Adapted code to use multiple style images instead of one
    
GlobalStyleImageCounter = 0
    
# Chooses a new style image every time.
# When all style images are used, it starts over
def make_input_batch_path(imgpath, bs, h, w, ps):
    images_path =[n for n in os.listdir(imgpath)]
    
    (_, _, c) = open_img(imgpath + images_path[0]).shape
    
    global GlobalStyleImageCounter
    
    bat = np.zeros(bs * h * w * c).astype(np.float32).reshape((bs, h, w, c))
    for i in range(bs):
        img = open_img(imgpath + STYLE_IMAGE_LIST[GlobalStyleImageCounter]) #random.randint(0,STYLE_IMAGE_COUNT-1)
        patch_fill(bat[i], img, 0, 0, 0, ps)
        
        # Update style image counter
        GlobalStyleImageCounter += 1
        if GlobalStyleImageCounter == STYLE_IMAGE_COUNT:
            GlobalStyleImageCounter = 0
    return bat


## Model
The model code is mostly unchanged. We removed the configuration for other image and patch sizes, since our images are always 256x256.

In [ ]:
def leaky_relu(x, lk = 0.2):
    return tf.maximum(x, x * lk)

def _fixed_padding(inputs, kernel_size, rate=1):
    kernel_size_effective = [kernel_size[0] + (kernel_size[0] - 1) * (rate - 1),
                            kernel_size[0] + (kernel_size[0] - 1) * (rate - 1)]
    pad_total = [kernel_size_effective[0] - 1, kernel_size_effective[1] - 1]
    pad_beg = [pad_total[0] // 2, pad_total[1] // 2]
    pad_end = [pad_total[0] - pad_beg[0], pad_total[1] - pad_beg[1]]
    padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg[0], pad_end[0]],
                        [pad_beg[1], pad_end[1]], [0, 0]], mode='SYMMETRIC')
    return padded_inputs


DL1C = 256
DL2C = 512

discriminator_cfg_p16 = {
    'l_num': 2,
    'l0_c': 3,                           # 256
    'l1_c': DL1C,   'l1_k': 4, 'l1_s': 4, # 64
    'l2_c': DL2C,   'l2_k': 4, 'l2_s': 4, # 16
}

batch_norm_decay=0.95
batch_norm_epsilon=0.001
batch_norm_updates_collections=tf.GraphKeys.UPDATE_OPS

def build_discriminator(inp, patch_size=16, is_training=True,
    name='discriminator', reuse=False):
    d_state = inp
    batch_norm_params = {
        'center': True,
        'scale': True,
        'decay': batch_norm_decay,
        'epsilon': batch_norm_epsilon,
        'updates_collections': batch_norm_updates_collections,
        'is_training': is_training
    }
    with slim.arg_scope([slim.batch_norm], **batch_norm_params):
        with tf.variable_scope(name, reuse=reuse):
            # conv s1
            cfg = discriminator_cfg_p16
            with slim.arg_scope([slim.conv2d],
                activation_fn=leaky_relu,
                normalizer_fn=slim.batch_norm,
                padding='VALID'):
                for l in range(1, cfg['l_num'] + 1):
                    d_state = slim.conv2d(d_state,
                        cfg['l%d_c'%l],
                        [cfg['l%d_k'%l], cfg['l%d_k'%l]],
                        stride=cfg['l%d_s'%l], scope='s1_%d'%l)
            d_state = slim.conv2d(d_state, 1, [1, 1], stride=1,
                activation_fn=tf.nn.sigmoid, scope='patch_mat')
    return d_state

g_encoder_cfg = {
    'l_num': 3,
    'l0_c': 32,   'l0_k': 3, 'l0_s': 2, # 1/2  --------> sc1
    'l1_c': 64,   'l1_k': 3, 'l1_s': 2, # 1/4  --------> sc2
    'l2_c': 128,  'l2_k': 3, 'l2_s': 2  # 1/8  --------> sc3 -L
}

g_residual_cfg = {
    'l_num': 1,
    'c': 128, 'k': 3
}

g_decoder_cfg = {
    'l_num': 3,
    'l0_c': 64,  'l0_k': 3, 'l0_s': 2, # --- x2
    'l1_c': 32,  'l1_k': 3, 'l1_s': 2, # --- x4
    'l2_c': 16,  'l2_k': 3, 'l2_s': 2, # --- x8
}

g_skip_conn_cfg = {
    'l_num' : 2
}

def build_generator(inp, name='generator', reuse=False):
    g_state = inp   # No prep
    with tf.variable_scope(name, reuse=reuse):
        cfg = g_encoder_cfg
        skip_conn = []
        with slim.arg_scope([slim.conv2d, slim.separable_conv2d], activation_fn=tf.nn.relu,
                    normalizer_fn=slim.instance_norm, padding='VALID'):
            for index in range(cfg['l_num']):
                g_state = _fixed_padding(g_state, [cfg['l%d_k'%index]])
                g_state = slim.separable_conv2d(g_state, None, [cfg['l%d_k'%index], cfg['l%d_k'%index]],
                    depth_multiplier=1, stride=cfg['l%d_s'%index], scope='enc_%d_dw'%index)
                g_state = slim.conv2d(g_state, cfg['l%d_c'%index], [1, 1], stride=1, scope='enc_%d_pw'%index)
                skip_conn.append(g_state)

        cfg = g_residual_cfg
        with slim.arg_scope([slim.conv2d, slim.separable_conv2d], activation_fn=tf.nn.relu,
                    normalizer_fn=slim.instance_norm, padding='VALID'):
            for index in range(cfg['l_num']):
                res_g = g_state
                g_state = _fixed_padding(g_state, [cfg['k']])
                g_state = slim.separable_conv2d(g_state, None, [cfg['k'], cfg['k']],
                    depth_multiplier=1, stride=1, scope='res_%d_dw'%index)
                g_state = slim.conv2d(g_state, cfg['c'], [1, 1], stride=1,
                    activation_fn=None, scope='res_%d_pw'%index)
                g_state = tf.nn.relu(g_state + res_g)

        cfg = g_decoder_cfg
        with slim.arg_scope([slim.conv2d, slim.separable_conv2d], activation_fn=None,
                    normalizer_fn=slim.instance_norm, padding='VALID'):
            for index in range(cfg['l_num']):
                g_state = tf.image.resize_images(g_state,
                    (g_state.shape[1]*2, g_state.shape[2]*2), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                g_state = _fixed_padding(g_state, [cfg['l%d_k'%index]])
                g_state = slim.separable_conv2d(g_state, None,
                    [cfg['l%d_k'%index], cfg['l%d_k'%index]], depth_multiplier=1, stride=1, scope='dec_%d_dw'%index)
                g_state = slim.conv2d(g_state, cfg['l%d_c'%index], [1, 1], stride=1, scope='dec_%d_pw'%index)
                # sc = slim.conv2d(skip_conn[?], 128, [1, 1], stride=1, scope='sc')
                if index < g_skip_conn_cfg['l_num']:
                    g_state = tf.nn.relu(g_state + skip_conn[g_skip_conn_cfg['l_num'] - index - 1])
                else:
                    g_state = tf.nn.relu(g_state)
        g_state = _fixed_padding(g_state, [3])
        g_state = slim.conv2d(g_state, 3, [3, 3], stride=1, padding='VALID',
            activation_fn=tf.nn.tanh, normalizer_fn=None, scope='output')
    return g_state


## VGG16
For calculating the content loss by performing image classification
Mostly unchanged. Except for a fix to an error while loading the weights file. We also removed some commented out code.

In [ ]:
# Required fix to properly load vgg16.npy weights
np_load_old = np.load # save np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k) # modify the default parameters of np.load

VGG_MEAN = [103.939, 116.779, 123.68]

class Vgg16:
    def __init__(self, vgg16_npy_path=None):
        self.data_dict = np.load("vgg16.npy", encoding='latin1').item()

    def build(self, rgb):
        """
        load variable from npy to build the VGG

        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """

        start_time = time.time()
        print("build model started")
        rgb_scaled = rgb * 255.0

        # Convert RGB to BGR
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
        
        bgr = tf.concat(axis=3, values=[
            blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
            red - VGG_MEAN[2],
        ])

        self.conv1_1 = self.conv_layer(bgr, "conv1_1")
        self.conv1_2 = self.conv_layer(self.conv1_1, "conv1_2")
      
        self.prob = self.conv1_2

    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def conv_layer(self, bottom, name):
        with tf.variable_scope(name):
            filt = self.get_conv_filter(name)

            conv = tf.nn.conv2d(bottom, filt, [1, 1, 1, 1], padding='SAME')

            conv_biases = self.get_bias(name)
            bias = tf.nn.bias_add(conv, conv_biases)

            relu = tf.nn.relu(bias)
            return relu

    def fc_layer(self, bottom, name):
        with tf.variable_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])

            weights = self.get_fc_weight(name)
            biases = self.get_bias(name)

            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)

            return fc

    def get_conv_filter(self, name):
        return tf.constant(self.data_dict[name][0], name="filter")

    def get_bias(self, name):
        return tf.constant(self.data_dict[name][1], name="biases")

    def get_fc_weight(self, name):
        return tf.constant(self.data_dict[name][0], name="weights")


## Training
Training the GAN on the style and photo images.
We modified this part to keep the TF session variable to prevent having to save it and load it again in the render part.
We also changed it to use our own multi-style batch generation function instead of the single style image one.

In [ ]:
input_ls = ls_files_to_json(TRAINSET_PATH, ext=['png', 'bmp', 'jpg', 'jpeg'])
TRAIN_SET = len(input_ls)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

input_s = tf.placeholder(tf.float32, shape=[BATCH_SIZE, PSI_D_SIZE, PSI_D_SIZE, 3], name='inps')
input_c = tf.placeholder(tf.float32, shape=[BATCH_SIZE, G_IMG_SIZE, G_IMG_SIZE, 3], name='inpc')

vgg_c = Vgg16()
with tf.name_scope("content_vgg"):
    vgg_c.build(input_c)

g_state = build_generator(input_c, name='generator')

vgg_g = Vgg16()
with tf.name_scope("content_vgg"):
    vgg_g.build(g_state)

dp_real = build_discriminator(input_s, patch_size=PATCH_SIZE, name='discriminator')
dp_fake = build_discriminator(g_state, patch_size=PATCH_SIZE, name='discriminator', reuse=True)

d_raw = vgg_c.prob # 128 * 128 * 64
d_gen = vgg_g.prob # 128 * 128 * 64

d_real_d = tf.reduce_mean(dp_real)
d_fake_d = tf.reduce_mean(dp_fake)

mean_d_fake = tf.reduce_mean(dp_fake)
d_fake_g = tf.reduce_mean((dp_fake) ** (1.0 - (dp_fake - mean_d_fake)))

d_loss = -(tf.log(d_real_d) + tf.log(1 - d_fake_d))
g_loss = (tf.norm(d_raw - d_gen) ** 2)*LAMBDA /(BATCH_SIZE*((G_IMG_SIZE/VGG_L)*(G_IMG_SIZE/VGG_L))*VGG_FEATURES)-tf.log(d_fake_g)

d_var_ls = tf.trainable_variables(scope='discriminator')
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step_d = tf.train.RMSPropOptimizer(5e-4).minimize(d_loss, var_list=d_var_ls)

g_var_ls = tf.trainable_variables(scope='generator')
train_step_g = tf.train.RMSPropOptimizer(5e-4).minimize(g_loss, var_list=g_var_ls)


sess.run(tf.global_variables_initializer())
var_ls = g_var_ls.append(d_var_ls)
epoch = 0

while epoch < TRAIN_EPOCHS:
    random.shuffle(input_ls)
    travx = int(TRAIN_SET / BATCH_SIZE) + (1 if (TRAIN_SET % BATCH_SIZE) != 0 else 0)
    for offset in range(travx):
        sub_ls  = pickup_list(input_ls, BATCH_SIZE, offset * BATCH_SIZE) # sub_ls  = pickup_list_random(input_ls, BATCH_SIZE)
        sub_img = images_batch(TRAINSET_PATH, sub_ls, prep=True,
                            shape=(G_IMG_SIZE, G_IMG_SIZE), singleCh=False, remove_pad=True)    
        for td in range(DISCRIMINATOR_MAX_ITER):
            sess.run(train_step_d, feed_dict={
                    input_s: make_input_batch_path(STYLE_PATH, BATCH_SIZE, PSI_D_SIZE, PSI_D_SIZE, PATCH_SIZE),
                    input_c: sub_img
                })
        for tg in range(GENERATOR_MAX_ITER):
            sess.run(train_step_g, feed_dict={
                    input_c: sub_img
                })
        # optional progress print
        #print('epoch %04d'%epoch, 'InnerProcess: %d/%d'%(offset, travx))


    cur_d_real = sess.run(d_real_d, feed_dict={
            input_s: make_input_batch_path(STYLE_PATH, BATCH_SIZE, PSI_D_SIZE, PSI_D_SIZE, PATCH_SIZE),
            input_c: sub_img
        })
    cur_d_fake = sess.run(d_fake_d, feed_dict={
            input_c: sub_img
        })
    print('\33[1;32mEpoch %d D_TURN D real\33[0m = '%epoch, cur_d_real.mean())
    print('\33[1;31mEpoch %d D_TURN D fake\33[0m = '%epoch, cur_d_fake.mean())
    epoch = epoch + 1

## Render
Use trained model to generate output images.
Instead of loading the saved model, we directly use the TF session of the previous code block.
This allows us to run the trained model without having to load and "warm up" the network, making the code a lot smaller and a bit more efficient.
However, this caused some issues with the batch size, so we included some fixes for that as well.

In [ ]:
if (not os.path.isdir(OUT_PATH)):
    os.makedirs(OUT_PATH)

input_ls = ls_files_to_json(IMGSRC_PATH, ext=['png', 'bmp', 'jpg', 'jpeg'])
CNT = len(input_ls)

for i in range(CNT):
    sub_ls  = pickup_list(input_ls, 1, i)
    #print("Processing:", sub_ls[0])
    sub_img, sls = imbatch_read(IMGSRC_PATH, sub_ls, (PSI_D_SIZE, PSI_D_SIZE))
    sub_img2  = np.tile(sub_img,(BATCH_SIZE, 1,1,1))
    time_start = time.time()
    render_batch = sess.run(g_state, feed_dict={input_c: sub_img2})
    img_write(render_batch[0],
            OUT_PATH+'/'+sub_ls[0],
        sls[0])# (PSI_D_SIZE, PSI_D_SIZE)) #

## Generate output images.zip and clean
Finally, we added these two lines to generate the images.zip file for submission and to clean all temp files from the project which we do not want to (or cannot, because of the large number of files) save.

In [ ]:
shutil.make_archive('/kaggle/working/images/', 'zip', 'images')
# remove all files except the output zip file
!rm -rf images preview vgg16.npy SUN* monet*